In [6]:
import pandas as pd
import numpy as np
import os
import re

# As a function

In [7]:
def interpolate(file_path):
    df = pd.read_csv(file_path, skiprows=[0,1])
    # skip first column
    df = df.iloc[:, 1:]

    skip = []
    next_id = []
    for col in range(2,25,3):
        for row in range(df.shape[0]):
            if row in skip:
                continue
            # previous threshold
            if df.iloc[row,col] < 0.8:
                prev_id = row-1
                # next threshold
                ct = 1
                while True:
                    try:
                        if df.iloc[row+ct,col] < 0.8:
                            skip.append(row+ct) # this row is consecutively < 0.8
                            ct+=1
                        else:
                            ls = [prev_id, row+ct, col-2, col-1]
                            next_id.append(ls)
                            break
                    except IndexError:
                        break

    df_new = df
    for i in next_id:
        gap = i[1] - i[0]
        x_diff = df_new.iloc[i[1], i[2]] - df_new.iloc[i[0], i[2]]
        y_diff = df_new.iloc[i[1], i[3]] - df_new.iloc[i[0], i[3]]
        # print(gap, x_diff, y_diff)
        x_step = x_diff / gap
        y_step = y_diff / gap
        for j in range(1,gap):
            df_new.iloc[i[0]+j, i[2]] = df_new.iloc[i[0], i[2]] + x_step*j
            df_new.iloc[i[0]+j, i[3]] = df_new.iloc[i[0], i[3]] + y_step*j
            # print(i[0]+j, i[2])
    
    return df_new

# Loop

In [8]:
folder = input("such as: Eng")

In [9]:
directory = "./CSVs/" + folder + '/'
csv_ls = os.listdir(directory)

In [10]:
for i in csv_ls:
    file = re.search('(Eng)?(Mand)?(_US)?\d{3}(_\d)?', i)[0]
    i_full = directory + i
    output = interpolate(i_full)
    # Save the updated DataFrame to a new CSV file or overwrite the existing one if needed
    output.to_csv('./normalized_csv/' + folder + '/' + file + '_interpolated.csv')